## Thay đổi batch size

In [ ]:
# Thay đổi batch_size, 64 - 128 - 256 - 512

batch_size = 256

## Chạy

In [ ]:
!pip install -q gdown
!gdown 115b2vVHryiPFOPYRumGBDIpx5dT_vH7R

Downloading...
From: https://drive.google.com/uc?id=115b2vVHryiPFOPYRumGBDIpx5dT_vH7R
To: /kaggle/working/aws_rainfall_all.parquet
100%|███████████████████████████████████████| 50.6M/50.6M [00:00<00:00, 152MB/s]


In [ ]:
import pandas as pd
from tqdm import tqdm
import gc
import numpy as np

pd.set_option("mode.chained_assignment", None)  # Disable caching
pd.options.display.memory_usage = False  # Reduce memory prints
gc.collect()

df = pd.read_parquet("/kaggle/working/aws_rainfall_all.parquet")

In [ ]:
def calculate_optimum_alpha(data):
    X_max = np.max(data)
    X_min = np.min(data)
    X_mean = np.mean(data)
    return ((X_max - X_min) - X_mean) / (X_max - X_min) if (X_max - X_min) != 0 else 0.5

def exponential_smoothing(data, alpha = 0.2):
    smoothed = np.zeros_like(data)
    smoothed[0] = data[0]
    for t in range(1, len(data)):
        smoothed[t] = alpha * data[t] + (1 - alpha) * smoothed[t-1]
    return smoothed

corr_matrix = df.corr()

threshold = 0.05

high_corr_features = corr_matrix["aws_rainfall"][abs(corr_matrix["aws_rainfall"]) > threshold].index.tolist()

high_corr_features = [i for i in high_corr_features if i not in ["Datetime", "Row", "Col"]]

def extract_to_numpy(df, n_steps = 6):
    feature_cols = high_corr_features
    X = []
    y = []
    positions = []
    for (row, col), group in tqdm(df.groupby(["Row", "Col"]), desc="Processing groups", unit="group"):
        group = group.sort_values("Datetime")
        group_values = group[feature_cols].values
        original_rainfall_values = group['aws_rainfall'].values
        alpha = calculate_optimum_alpha(original_rainfall_values)
        # alpha = 0.5
        rainfall_values = exponential_smoothing(original_rainfall_values, alpha)
        for i in range(n_steps, len(group_values)):
            if(len(group_values) < n_steps):
                break
            X.append(group_values[i - n_steps:i])
            y.append(rainfall_values[i])
            positions.append((row, col, group.iloc[i]["Datetime"]))
    return np.array(X), np.array(y), np.array(positions)

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

columns = [i for i in high_corr_features if i != "aws_rainfall"]
df_feat = df[columns].replace([9999, np.inf, -np.inf], np.nan)

scaler = StandardScaler()
df_feat_scaled = pd.DataFrame(scaler.fit_transform(df_feat), columns = columns)

imputer = KNNImputer(n_neighbors = 5, weights = "distance")
df_feat_imputed_scaled = pd.DataFrame(imputer.fit_transform(df_feat_scaled), columns = columns)

df_feat_imputed = pd.DataFrame(scaler.inverse_transform(df_feat_imputed_scaled), columns = columns)

In [ ]:
new_df = pd.concat([df[["Datetime", "Row", "Col", "aws_rainfall"]], df_feat_imputed], axis=1)
del df_feat, df_feat_scaled, df_feat_imputed_scaled, df_feat_imputed

In [ ]:
def data_smoothing(df):
    feature_cols = [col for col in df.columns if col not in ["Row", "Col", "Datetime", "aws_rainfall", "smoothed_rainfall"]]

    for (row, col), group in tqdm(df.groupby(["Row", "Col"]), desc="Processing groups", unit="group"):
        group = group.sort_values("Datetime")
        original_rainfall_values = group['aws_rainfall'].values
        alpha = calculate_optimum_alpha(original_rainfall_values)
        alpha = 0.5
        rainfall_values = exponential_smoothing(original_rainfall_values, alpha)

        # Update smoothed rainfall values back to the DataFrame
        df.loc[group.index, 'smoothed_rainfall'] = rainfall_values

    return df

In [ ]:
new_df = data_smoothing(new_df)

Processing groups: 100%|██████████| 334/334 [00:03<00:00, 86.92group/s]


In [ ]:
x, y, z = extract_to_numpy(new_df)

Processing groups: 100%|██████████| 334/334 [00:58<00:00,  5.68group/s]


In [ ]:
print("Shape of X:", x.shape)
print("Shape of y:", y.shape)
print("Shape of positions:", z.shape)

Shape of X: (935534, 6, 13)
Shape of y: (935534,)
Shape of positions: (935534, 3)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Initialize scalers
rainfall_scaler = MinMaxScaler()  # For past rainfall (first feature) & y
features_scaler = MinMaxScaler()  # For other features

# Fit rainfall scaler on both past rainfall and y
rainfall_scaler.fit(np.concatenate([y.reshape(-1, 1), x[:, :, 0].reshape(-1, 1)]))

# Transform y
y_scaled = rainfall_scaler.transform(y.reshape(-1, 1)).flatten()

# Copy x to avoid modifying original data
x_scaled = np.copy(x)

# Scale only the first feature (past rainfall) using rainfall_scaler
x_scaled[:, :, 0] = rainfall_scaler.transform(x[:, :, 0].reshape(-1, 1)).reshape(x.shape[0], x.shape[1])

# Get the actual number of features dynamically
num_features = x.shape[2] - 1  # Exclude the first feature

# Scale the remaining features
x_scaled[:, :, 1:] = features_scaler.fit_transform(x[:, :, 1:].reshape(-1, num_features)).reshape(x.shape[0], x.shape[1], num_features)

# Function to inverse transform y predictions
def inverse_transform_y(y_pred_scaled):
    return rainfall_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Train-test split
# X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(x_scaled, y_scaled, z, test_size=0.2, random_state=42)

split_index = int(len(x_scaled) * 0.8)  

# Training set (first 80%)
X_train, y_train, z_train = x_scaled[:split_index], y_scaled[:split_index], z[:split_index]

# Testing set (last 20%)
X_test, y_test, z_test = x_scaled[split_index:], y_scaled[split_index:], z[split_index:]

# Reshape z_train and z_test
z_train = z_train.reshape(-1, 3)
z_test = z_test.reshape(-1, 3)

# Ensure y_train and y_test remain 1D
y_train = y_train.flatten()
y_test = y_test.flatten()


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

n_steps, num_features = x.shape[1], x.shape[2]

def build_model(hp):
    # Hyperparameters
    lstm_units_1 = hp.Int('lstm_units_1', min_value=32, max_value=256, step=32)
    lstm_units_2 = hp.Int('lstm_units_2', min_value=16, max_value=128, step=16)
    lstm_units_3 = hp.Int('lstm_units_3', min_value=8, max_value=64, step=8)

    dropout_rate_1 = hp.Float('dropout_rate_1', min_value=0.0, max_value=0.5, step=0.1)
    dropout_rate_2 = hp.Float('dropout_rate_2', min_value=0.0, max_value=0.5, step=0.1)
    dropout_rate_3 = hp.Float('dropout_rate_3', min_value=0.0, max_value=0.5, step=0.1)

    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    # Create model using Functional API
    inputs = Input(shape=(n_steps, num_features))

    # First LSTM layer
    lstm1 = LSTM(lstm_units_1, activation='relu', return_sequences=True)(inputs)
    bn1 = BatchNormalization()(lstm1)
    drop1 = Dropout(dropout_rate_1)(bn1)

    # Second LSTM layer
    lstm2 = LSTM(lstm_units_2, activation='relu', return_sequences=True)(drop1)
    bn2 = BatchNormalization()(lstm2)
    drop2 = Dropout(dropout_rate_2)(bn2)

    # Third LSTM layer
    lstm3 = LSTM(lstm_units_3, activation='relu')(drop2)
    bn3 = BatchNormalization()(lstm3)
    drop3 = Dropout(dropout_rate_3)(bn3)

    # Output layer
    outputs = Dense(1, activation='linear')(drop3)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

# Set up the Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=64,
    factor=3,
    directory='keras_tuner_dir',
    project_name='lstm_tuning'
)

tuner.search_space_summary()

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

# Start the hyperparameter search
tuner.search(
    X_train, y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    batch_size=batch_size
)

Trial 90 Complete [00h 10m 33s]
val_loss: 0.0001391547848470509

Best val_loss So Far: 0.00013905524974688888
Total elapsed time: 04h 14m 19s


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best LSTM units 1: {best_hps.get('lstm_units_1')}")
print(f"Best LSTM units 2: {best_hps.get('lstm_units_2')}")
print(f"Best LSTM units 3: {best_hps.get('lstm_units_3')}")
print(f"Best dropout rate 1: {best_hps.get('dropout_rate_1')}")
print(f"Best dropout rate 2: {best_hps.get('dropout_rate_2')}")
print(f"Best dropout rate 3: {best_hps.get('dropout_rate_3')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

model = tuner.hypermodel.build(best_hps)

history = model.fit(
    X_train, y_train,
    epochs=64,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")

Best LSTM units 1: 32
Best LSTM units 2: 80
Best LSTM units 3: 32
Best dropout rate 1: 0.2
Best dropout rate 2: 0.0
Best dropout rate 3: 0.4
Best learning rate: 0.0003424899128529849
Epoch 1/64
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - loss: 0.6097 - mae: 0.4815 - val_loss: 2.5004e-04 - val_mae: 0.0041
Epoch 2/64
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 3.7225e-04 - mae: 0.0082 - val_loss: 2.4137e-04 - val_mae: 0.0032
Epoch 3/64
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 2.2756e-04 - mae: 0.0038 - val_loss: 2.2435e-04 - val_mae: 0.0030
Epoch 4/64
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 2.0767e-04 - mae: 0.0038 - val_loss: 1.9824e-04 - val_mae: 0.0027
Epoch 5/64
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 2.0101e-04 - mae: 0.0037 - val_loss: 1.7821e-04 - val_mae: 0.0025
Epoch 6/64
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 1.8222e-04 - mae: 0.0036 - val_loss: 3.0761e-04 - val_mae: 0.0026
Epoch 7/64
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 12s 4

## Metrics

In [ ]:
y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

5848/5848 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
MAE: 0.0023
MSE: 0.0001
RMSE: 0.0118
R² Score: 0.4609


## So sánh

In [ ]:
import matplotlib.pyplot as plt

y_pred = model.predict(X_test)

plt.figure(figsize=(10, 5))
plt.plot(y_test, label="Actual")
plt.plot(y_pred, label="Predicted", linestyle="dashed")
plt.legend()
plt.show()
